# DTEK0042 Exercise 2
    Group Members:
    1. Tiina Nokelainen
    2. Risto Hirvilammi
    3. Oskari Läntinen

*** Note for Google Colab Users ***
     Because Google Colab doesn't have a button to convert your notebook to .html format here is a quick work around:
   1. open a new google colab notebook
   2. in the files section in google colab, upload the .ipynb file you want to be converted to .html
   3. in your new open notebook run this command in an empty cell: !jupyter nbconvert --to html YourFileName.ipynb
   4. after the command is finished running refresh the page
   5. In your files section you should see your original .ipynb file that you uploaded and then a .html file of that same notebook.
   6. download the .html file and you're good to go!

In this exercise, you are required to analyze an ECG signal step-by-step as outlined below. The deliverables for this exercise are a jupyter notebook and a .html file exported form the notebook. The notebook should includes your code, observations, graphs, and conclusions made upon analyzing the given ECG signal. Please provide caption and description for every figure. 

## 1- library Imports

In [ ]:
import scipy
import numpy as np
import matplotlib.pyplot as plt

# 2- Data Import and plotting
* Import the ECG signal named “ECG_800Hz.txt” into your python environment and store it in a variable named “ECG_sig”. 
* Plot the signal.
* Note: the sampling frequency of this signal is 800Hz. 
* You need this value if you want to plot ECG versus time.


     HINT: 
         ECG_sig = np.loadtxt(the directory of the ECG signal) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Loading the signal to colab
ECG_signal = np.loadtxt("/content/drive/My Drive/Biosignals/ECG_800hz.txt")
print("Done!")

Done!


In [ ]:
# Create the array for time and plot signal vs time
time=np.linspace(0,(len(ECG_signal)/800),len(ECG_signal))

plt.figure(figsize=(20,7))
plt.plot(time,ECG_signal)
plt.xlabel('Time (s)')
plt.ylabel('Signal (au)')
plt.title("ECG_signal vs time")

**ECG_signal vs time (s):**
Shows the original, unfiltered, ECG signal



# 3- Discrete Fourier Transform

 * Compute DFT using FFT algorithm provided by scipy package. Take only the positive frequencies from the computed DFT and subsequently calculate the magnitude of frequency content. 
 * Plot the calculated magnitude versus frequency.
 * Discard the frequencies below 0.5Hz and above 40Hz and replot the magnitude versus frequency.
 * Note: depending on the length of the signal the last positive frequency is placed either at an odd or even index in the computed DFT array. When you consider only positive frequencies the minimum frequency in your DFT is DC frequency or f = 0Hz and the maximum frequency is Nyquist frequency. 
 https://scipy.github.io/devdocs/tutorial/fft.html 
 
      
      HINT: 
        ECG_sig_DFT = scipy.fft (ECG_sig)  
        frequencies = np.fft.fftfreq(len(ECG_sig)) 

In [ ]:
from scipy.fft import rfft, rfftfreq

In [ ]:

### Fourier transform ###
Fs=800 # Sampling rate

ecg_dft = rfft(ECG_signal)
freqs = rfftfreq(len(ECG_signal), 1/Fs)

plt.figure(figsize=(12,7))
plt.plot(freqs[(freqs > 0.5) & (freqs < 40)], np.abs(ecg_dft[(freqs > 0.5) & (freqs < 40)]))
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude");

# 4- Band Pass Filter Design
*  Design a band pass filter. Use Butterworth filter of order 4 with cutoff frequencies equal to 0.5Hz and 40Hz.


* Design another butterworth band pass filter but this time use an order of 2 with cutoff frequencies equal to 0.5Hz and 40Hz


* Plot the frequency response of both filters and explain your observations.


* Note: the documentation of scipy package online is very comprehensive and informative. there are examples that 
  you can easilty follow and use to solve the given exercise here. 
  https://docs.scipy.org/doc/scipy-0.14.0/reference/signal.html    
    
   
      HINT: 
        from scipy.signal import butter, filtfilt, freqz
    

In [ ]:
from scipy import signal

In [ ]:
# Butterworth order of 4
b, a = signal.butter(4, [0.5, 40], 'band', analog=True)
w, h = signal.freqs(b, a)
plt.plot(w, 20 * np.log10(abs(h)))
plt.xscale('log')
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(0.5, color='green') # low cutoff frequency
plt.axvline(40, color='green') # high cutoff frequency
plt.show()

In [ ]:
# Butterworth order of 2
b, a = signal.butter(2, [0.5, 40], 'band', analog=True)
w, h = signal.freqs(b, a)
plt.plot(w, 20 * np.log10(abs(h)))
plt.xscale('log')
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(0.5, color='green') # low cutoff frequency
plt.axvline(40, color='green') # high cutoff frequency
plt.show()

## Observations:
  With butterworth order 4 the cutoff is more steep than with order 2.

In [ ]:
def bandPassFilter(signal, order):

  fs = 800
  lowcut = 0.5
  highcut = 40
  
  # maximum frequency is Nyquist frequency which his half of the sampling freq
  nyq = 0.5 * fs
  low = 0.5 / nyq
  high = 40 / nyq

  b, a = scipy.signal.butter(order, [low, high], 'band', analog=False)
  filt_signal = scipy.signal.filtfilt(b, a, signal, axis=0)

  return filt_signal


# 5- Apply filter to ECG_Sig
* Use the designed 4th order filter to filter ECG_sig.
* Plot the first 5000 samples from the original signal (raw_signal) and the filtered signal.
* What happens after filtering? Explain your observations.

In [ ]:
filtered_signal = bandPassFilter(ECG_signal, 4)


In [ ]:

fig = plt.figure(figsize=(12,7)) 
plt.plot(ECG_signal[:5000], label="raw signal", linewidth=1)
plt.plot(filtered_signal[:5000], label="filtered signal", linewidth=1)
fig.legend(bbox_to_anchor=(0.20,0.8))
plt.xlabel('Time (?)')
plt.ylabel('ECG signal (au)')
plt.title("Effect of filtering on ECG signal vs time")

## Observations: 
    Noise from the power supply (50 Hz) has reduced and other higher frequency noise signals.
  
  

# 6- QRS Detection 
*  QRS detection using “hamilton” method provided by the “biosppy” package.
*  If you do not have this package installed, use the following command to install it in your anaconda prompt: 
    pip install biosppy
* You can also use following command in your notebook: !pip install biosppy
* Plot the results and describe your observations about QRS and Heart rate.

* See the links below for more help:
https://pypi.org/project/biosppy/  
https://biosppy.readthedocs.io/en/stable/index.html#simple-example 

      HINT: 
        import biosppy 
        from biosppy.signals import ecg 



In [ ]:
!pip install biosppy

In [ ]:
from biosppy.signals import ecg

In [ ]:
r_peaks = ecg.hamilton_segmenter(filtered_signal, sampling_rate=800)

fig = plt.figure(figsize=(20,9))
plt.plot(filtered_signal, linewidth=0.5)
plt.plot(r_peaks['rpeaks'], filtered_signal[r_peaks['rpeaks']], "x");
plt.xlabel('Time (?)')
plt.ylabel('ECG signal (au)')
plt.title("ECG signal vs time (peaks marked)")

In [ ]:
r_peaks['rpeaks'].shape

In [ ]:
ecg_cut = filtered_signal[:20000]
cut_rpeaks = ecg.hamilton_segmenter(ecg_cut, sampling_rate=800)['rpeaks']

fig = plt.figure(figsize=(14,7))
plt.plot(ecg_cut, linewidth=1)
plt.plot(cut_rpeaks, ecg_cut[cut_rpeaks], "x");
plt.xlabel('Time (?)')
plt.ylabel('ECG signal (au)')
plt.title("Filtered ECG signal vs time (peaks marked)")

## Observations:
    Nice looking ECG after filterinh. R-peaks are well detected.
    